# Classificação faixa TRL -  GPT-2 + 512 tokens + Modelos ML


- https://huggingface.co/pierreguillou/gpt2-small-portuguese

## Código Versão 1.0 - 26 OUT 23

- Uso dos classificadores: *Modelos de ML*
- Matriz de confusão;
- Semente 42

- Esse código adota a simplificação proposta na dissertação, no sentido de obtermos a RV, depois alizarmos o k-fold. Dessa forma, temos uma econnomia computacional.

In [ ]:
# dataset.csv   ou  dataset_pre_processado_1.csv  ou  dataset_pre_processado_stem_2.csv
#     CSV1                  CSV2                                   CSV3
dataset = "dataset.csv"

In [ ]:
print("Lembre-se estamos usando o dataset: " + dataset)

Lembre-se estamos usando o dataset: dataset.csv


In [ ]:
melhor_modelo = 'best_model_TRL_gpt2_ml_' + '.bin'
melhor_modelo

'best_model_TRL_gpt2_ml_.bin'

- Quantidade de tokens máxima = 512

In [ ]:
MAX_LEN = 512

## Preparação

### Bibliotecas e ambiente

In [ ]:
!pip install -qq transformers

In [ ]:
!pip install -q -U watermark

In [ ]:
!nvidia-smi

Fri Oct 27 04:05:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    31W /  70W |   2095MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from torch import nn
from torch.optim import Adam
from transformers import GPT2Model, GPT2Tokenizer

In [ ]:
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from collections import defaultdict
import textwrap

from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes, svm
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

numpy       : 1.23.5
pandas      : 1.5.3
torch       : 2.1.0+cu118
transformers: 4.34.1



# Particoes KFOLD

- Para mais detalhes, consulte o código *1-kfold.ipynb* em que as partições foram sorteadas. As células a seguir recriam o resultado obtido.

In [ ]:
array1 = np.array([  0,   1,   2,   3,   5,   7,   9,  10,  11,  12,  13,  14,  15,
         16,  17,  19,  20,  21,  22,  23,  25,  26,  27,  28,  30,  32,
         33,  34,  35,  36,  37,  39,  40,  44,  45,  46,  47,  48,  49,
         50,  52,  53,  54,  55,  56,  57,  59,  60,  61,  63,  64,  66,
         68,  69,  70,  71,  72,  75,  76,  77,  78,  79,  80,  81,  82,
         83,  84,  85,  87,  91,  92,  93,  94,  96,  97,  98,  99, 100,
        102, 103, 104, 105, 106, 108, 109, 110, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130,
        131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145,
        146, 148, 150, 151, 153, 154, 155, 157, 158, 159, 160, 161, 163,
        164, 165, 166, 167])

array2 = np.array([  0,   1,   3,   4,   5,   6,   7,   8,  10,  13,  14,  15,  16,
         17,  18,  19,  20,  22,  23,  24,  25,  26,  27,  29,  30,  31,
         34,  36,  37,  38,  40,  41,  42,  43,  44,  45,  47,  49,  51,
         52,  53,  54,  55,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  73,  74,  75,  76,  77,  80,  83,
         84,  86,  88,  89,  90,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102, 104, 105, 107, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 132, 133, 135, 136, 137, 138, 139, 142, 143, 144, 145,
        147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 161, 162,
        163, 165, 166, 167])
array3 = np.array([  1,   2,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         17,  18,  20,  21,  22,  23,  24,  25,  28,  29,  30,  31,  32,
         33,  35,  36,  37,  38,  39,  41,  42,  43,  44,  45,  46,  47,
         48,  50,  51,  52,  54,  55,  56,  57,  58,  59,  62,  63,  65,
         67,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
         82,  84,  85,  86,  87,  88,  89,  90,  91,  93,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111,
        112, 113, 114, 118, 120, 121, 122, 123, 124, 126, 128, 129, 131,
        132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 144, 146, 147,
        148, 149, 150, 151, 152, 153, 155, 156, 158, 159, 160, 161, 162,
        163, 164, 165, 166])
array4 = np.array([  0,   2,   3,   4,   6,   8,   9,  10,  11,  12,  15,  16,  18,
         19,  20,  21,  22,  24,  25,  26,  27,  28,  29,  31,  32,  33,
         34,  35,  37,  38,  39,  40,  41,  42,  43,  45,  46,  47,  48,
         49,  50,  51,  53,  56,  57,  58,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  71,  72,  73,  74,  75,  76,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
         94,  95,  98, 100, 101, 103, 104, 106, 107, 108, 109, 110, 111,
        112, 114, 115, 116, 117, 119, 121, 123, 124, 125, 127, 128, 129,
        130, 131, 133, 134, 135, 137, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159,
        162, 163, 164, 166, 167])
array5 = np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  21,  23,  24,  26,  27,  28,  29,
         30,  31,  32,  33,  34,  35,  36,  38,  39,  40,  41,  42,  43,
         44,  46,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,
         60,  61,  62,  64,  65,  66,  67,  68,  69,  70,  72,  73,  74,
         77,  78,  79,  81,  82,  83,  85,  86,  87,  88,  89,  90,  91,
         92,  94,  95,  96,  97,  99, 101, 102, 103, 105, 106, 107, 108,
        110, 111, 113, 115, 116, 117, 118, 119, 120, 122, 125, 126, 127,
        128, 130, 131, 132, 133, 134, 136, 137, 138, 140, 141, 143, 144,
        145, 146, 147, 148, 149, 151, 152, 154, 156, 157, 158, 159, 160,
        161, 162, 164, 165, 167])
# Criar a lista composta
kfold_train = [array1, array2, array3, array4, array5]

In [ ]:
# Criar as cinco arrays numpy
array1 = np.array([  4,   6,   8,  18,  24,  29,  31,  38,  41,  42,  43,  51,  58, 62,  65,  67,  73,  74,  86,  88,  89,  90,  95, 101, 107, 111, 128, 133, 144, 147, 149, 152, 156, 162])
array2 = np.array([  2,   9,  11,  12,  21,  28,  32,  33,  35,  39,  46,  48,  50, 56,  72,  78,  79,  81,  82,  85,  87,  91, 103, 106, 108, 131, 134, 140, 141, 146, 148, 158, 159, 164])
array3 = np.array([  0,   3,  15,  16,  19,  26,  27,  34,  40,  49,  53,  60,  61, 64,  66,  68,  69,  83,  92,  94, 110, 115, 116, 117, 119, 125, 127, 130, 137, 143, 145, 154, 157, 167])
array4 = np.array([  1,   5,   7,  13,  14,  17,  23,  30,  36,  44,  52,  54,  55, 59,  70,  77,  96,  97,  99, 102, 105, 113, 118, 120, 122, 126, 132, 136, 138, 151, 160, 161, 165])
array5 = np.array([ 10,  20,  22,  25,  37,  45,  47,  57,  63,  71,  75,  76,  80,  84,  93,  98, 100, 104, 109, 112, 114, 121, 123, 124, 129, 135, 139, 142, 150, 153, 155, 163, 166])


# Criar a lista composta
kfold_test = [array1, array2, array3, array4, array5]

# Dataset

In [ ]:
df = pd.read_csv(dataset)

In [ ]:
novo_df = df[["resumo", "rotulo"]]

In [ ]:
def adapto_faixa(faixa):
    faixa -=1
    return faixa

In [ ]:
novo_df['rotulo'] = novo_df.rotulo.apply(adapto_faixa)

<ipython-input-84-d13f2641b9c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df['rotulo'] = novo_df.rotulo.apply(adapto_faixa)


In [ ]:
novo_df.tail()

,resumo,rotulo
163,Rio de Janeiro (RJ) – O Centro de Avaliações d...,2
164,Este trabalho apresenta um sistema para contro...,2
165,No contexto das comunicações táticas baseadas ...,2
166,O valor da velocidade de alvos móveis em image...,0
167,Neste artigo é apresentada a análise da seção ...,0


In [ ]:
class_names = ['Faixa 1', 'Faixa 2', 'Faixa 3']

### gpt2-small-portuguese

- https://huggingface.co/pierreguillou/gpt2-small-portuguese

```
@inproceedings{pierre2020gpt2smallportuguese,
  title={GPorTuguese-2 (Portuguese GPT-2 small): a Language Model for Portuguese text generation (and more NLP tasks...)},
  author={Pierre Guillou},
  year={2020}
}

```


- **Tokenizer**: No GPT-2, ao contrário do BERT, o preenchimento é feito à esquerda, uma vez que o último token é utilizado para a previsão.

In [ ]:
PRE_TRAINED_MODEL_NAME = "pierreguillou/gpt2-small-portuguese"
# tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer = GPT2Tokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
# tokenizer.model_max_length=MAX_LEN
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = GPT2Model.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
frase = 'A avaliação de prontidão tecnológica em tecnologias de interesse militar'

In [ ]:
gpt2_input = tokenizer(frase, padding="max_length", max_length=16, truncation=True, return_tensors="pt")

In [ ]:
print(gpt2_input['input_ids'])
print(gpt2_input["attention_mask"])

tensor([[    0,     0,     0,     0,     0,    33,  7912,   261,   374, 38198,
         20142,   300,  9643,   261,  3325,  2303]])
tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
example_text = tokenizer.decode(gpt2_input.input_ids[0])
print(example_text)

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>A avaliação de prontidão tecnológica em tecnologias de interesse militar


In [ ]:
encoding = tokenizer.encode_plus(
  frase,
  max_length=20,   #Perceba que irei cortar um token da minha frase anterior
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


dict_keys(['input_ids', 'attention_mask'])

- Informações sobre o modelo:

In [ ]:
model = model.to(device)

In [ ]:
model.config.hidden_size

768

In [ ]:
model.config

GPT2Config {
  "_name_or_path": "pierreguillou/gpt2-small-portuguese",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.34.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [ ]:
# https://medium.com/@TaaniyaArora/visualizing-gpt2-word-embeddings-on-tensorboard-ea5c8fef9efa
# gpt2_word_embeddings = model.wte.weight # Word Token Embeddings
# print(gpt2_word_embeddings.shape)
# gpt2_word_embeddings
# gpt2_word_embeddings = model.transformer.wte.weight # Word Token Embeddings
# print(gpt2_word_embeddings.shape)

# Aplicando o Modelo para gerar a representação vetorial


- Dúvidas e explicações sobre o GPT-2 para Feature Extraction
```
1 - https://stackoverflow.com/questions/60574112/can-we-use-gpt-2-sentence-embedding-for-classification-tasks
```
```
2 - https://github.com/huggingface/transformers/issues/3168
```



- Não encontrei um tutorial que fosse propriamente para o GPT-2, mas foi possível adaptar os tutoriais de BERT para ele, devido as similaridades das arquiteturas baseadas em *transformers*.

- Tutorial interessante:

```
1 - https://towardsdatascience.com/feature-extraction-with-bert-for-text-classification-533dde44dc2f
```

```
2 - https://github.com/felipemaiapolo/legalnlp/blob/main/demo/BERT/BERT_TUTORIAL.ipynb
```

*Recomendo muito o 2, pois foi feito em formato de notebook, bem redigido, comentando os passos e explicando o BERT. ALém de estar em português (material raro em termos de qualidade em nossa língua)*

- Um cuidado que se deve ter é em relação a máscara de atenção, em ambos tutoriais, ela não era passada, o que gerava um ALERTA. Consultando a docuemntação, vi como deve ser passado, e ao final, os modelos tiveram um ganho de desempenho.

### Transformando os dados em tensores:

In [ ]:
# Fazendo a padronização dos textos
wrapper = textwrap.TextWrapper()
data_resumo = list(novo_df['resumo'])

- Visualizando:

In [ ]:
for text in range(len(data_resumo[:2])):
  print(f'{wrapper.fill(data_resumo[text])}')
  print()

O crescente emprego de mísseis de ombro infravermelhos contra alvos
aéreos demanda a utilização de contramedidas cada vez mais modernas e
eficientes. Neste cenário, surge o Directed Infrared Countermeasure
(DIRCM), cujo objetivo é interferir no guiamento do míssil por meio de
pulsos de laser. Neste artigo, um seeker infravermelho do tipo rising
sun é modelado e simulado, sendo os efeitos da emissão de um DIRCM no
processamento do sinal avaliados. A influência de parâmetros de
frequência de repetição de pulsos e intensidade do laser são
evidenciados. Os resultados obtidos ressaltam a importância e a
necessidade do desenvolvimento de ferramentas computacionais mais
complexas, visando ao desenvolvimento da doutrina de emprego deste
tipo de contramedida.

Materiais dielétricos com baixas perdas e alta permissividade são
componentes essenciais que são utilizados em linhas de transmissões
não lineares capacitivas (LTNLs) na geração de RF. LTNLs possuem
grande potencial para gerar ondas de só

In [ ]:
# Aplicando o bert_tokenizer com o comprimento máximo que definimos
encoded_inputs = tokenizer(data_resumo, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")


#O encoded_input está como um dicionário com 3 chaves
encoded_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

- Visualizando como o resumo ficou tokenizado, i.e., a chave ''input_ids' do encoded_inputs :

In [ ]:
#Visualiando o primeiro texto após a aplicação do tokenizador
print(encoded_inputs['input_ids'][0])

print()

# Mostrando o mesmo texto decodificado
print(wrapper.fill(tokenizer.decode(encoded_inputs['input_ids'][0])))

tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
print(encoded_inputs['input_ids'].shape)  # batch size x seq length
print(encoded_inputs['attention_mask'].shape)

torch.Size([168, 512])
torch.Size([168, 512])


In [ ]:
# Passando os tensores para para a GPU - aceleração através da GPU
input_ids = encoded_inputs['input_ids'].to(device)
attention_mask = encoded_inputs['attention_mask'].to(device)
# encoded_inputs = encoded_inputs.to(device)


# input_ids = encoded_inputs['input_ids']
# attention_mask = encoded_inputs['attention_mask']

In [ ]:
# Criando o vetor de features/caracteristicas
features = []

- buscando entender sobre a classe model e o seu retorno

In [ ]:
# type(model(input_ids[0:(1)]))

In [ ]:
# type(model(input_ids[0:(1)])[1])

In [ ]:
# len(model(input_ids[0:(1)])[1])

In [ ]:
# model(input_ids[0:(1)]).last_hidden_state

In [ ]:
# model(input_ids[0:(1)]).last_hidden_state.cpu

In [ ]:
# model(**encoded_inputs[0:(1)]).last_hidden_state

- Aplicando o GPT-2 para a extração das características

Documentação oficial que precisei estudar, para ocnseguir adaptar o processo do BERT para o GPT-2:

```
1 - https://huggingface.co/transformers/v4.9.2/model_doc/gpt2.html#gpt2model
```



In [ ]:
# for i in tqdm(range(len(data_resumo))):
#     with torch.no_grad():
#       last_hidden_states = model(input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
#     features.append(last_hidden_states)

for i in tqdm(range(len(data_resumo))):
    with torch.no_grad():
      # last_hidden_states = model(input_ids[i:(i+1)],attention_mask[i:(i+1)]).last_hidden_state.cpu().numpy().reshape(-1).tolist()
      last_hidden_states = model(input_ids[i:(i+1)]).last_hidden_state.cpu().numpy().reshape(-1).tolist()
      # last_hidden_states = model(**encoded_inputs).last_hidden_state.cpu().numpy().reshape(-1).tolist()
    features.append(last_hidden_states)

100%|██████████| 168/168 [00:10<00:00, 16.03it/s]


In [ ]:
# passando a lista features para numpy array com as features extraidas
features = np.array(features)

- Visualizando o resultado:

In [ ]:
print('FEATURES: Número de linhas: ' + str(features.shape[0]) + ' Número de colunas: ', str(features.shape[1]))
print("FEATURES é um objeto " + str(type(features)))

FEATURES: Número de linhas: 168 Número de colunas:  393216
FEATURES é um objeto <class 'numpy.ndarray'>


In [ ]:
features[0]

array([-0.19670489,  0.17107035,  0.10749422, ...,  0.28390408,
        0.2394657 ,  0.86860579])

# Adaptando a representação vetorial gerada

- Perceba que até aqui, temos um numpy array chamado **features**, com 168 linhas e 768 colunas.

- Iremos transformar nosso numpy array em um dataframe novamente:

In [ ]:
df_repvet = pd.DataFrame(features)
repvet_rotulado = pd.concat([df_repvet, novo_df['rotulo']], axis = 1)
repvet_rotulado.shape

(168, 393217)

In [ ]:
df_repvet

,0,1,2,3,4,5,6,7,8,9,...,393206,393207,393208,393209,393210,393211,393212,393213,393214,393215
0,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.253514,-0.368506,0.452481,-0.077948,-0.174519,0.397721,0.439606,0.283904,0.239466,0.868606
1,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.103246,-0.352199,0.157616,-0.666430,-0.755236,0.441820,0.579303,0.448379,0.324226,1.281295
2,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,-0.076338,-1.457131,0.513621,-0.785321,-0.698155,-0.144908,-0.032716,0.113876,0.423749,0.794382
3,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.208831,-1.140846,-0.148798,-1.034243,-0.268602,0.026068,0.578380,0.321614,0.406567,0.325306
4,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.381602,-0.865939,0.340151,-0.084570,-0.026963,0.801974,0.644628,0.329671,0.112665,0.851061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.218819,-0.662356,-0.900760,0.443632,-1.395365,0.030917,0.001971,1.138035,0.115485,1.609506
164,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.628640,0.706906,0.023609,0.772302,-1.505951,-0.658736,-0.689788,1.544010,-0.677629,0.837013
165,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.415427,-0.300990,0.774251,-0.609041,-0.432964,0.936262,0.529477,0.179168,-0.028599,0.790166
166,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.078053,-0.565056,0.281885,-0.548133,-0.682248,-0.154583,0.609861,0.535134,0.148935,0.412213


In [ ]:
repvet_rotulado.head(3)

,0,1,2,3,4,5,6,7,8,9,...,393207,393208,393209,393210,393211,393212,393213,393214,393215,rotulo
0,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,-0.368506,0.452481,-0.077948,-0.174519,0.397721,0.439606,0.283904,0.239466,0.868606,0
1,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,-0.352199,0.157616,-0.666430,-0.755236,0.441820,0.579303,0.448379,0.324226,1.281295,1
2,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,-1.457131,0.513621,-0.785321,-0.698155,-0.144908,-0.032716,0.113876,0.423749,0.794382,0


# Criando uma estrutura para armazenar os resultados

In [ ]:
# Algoritmos de classificação
classifiers = [  #NB, KNN, SVM
    ('Multinomial Naive Bayes', MultinomialNB()),
    ('Complement Naive Bayes Classifier', ComplementNB()),
    ('KNN', KNeighborsClassifier()),  #n_neighbors default é 5
    ('SVM', svm.SVC( )),
     ('Random Forest', RandomForestClassifier(random_state=42)),
      ('AdaBoost',    AdaBoostClassifier(random_state=42)) #n_estimators default é 50
]

In [ ]:
lista_classificador_nome = list()
for classifier_name, classifier in classifiers:
    lista_classificador_nome.append(classifier_name)

In [ ]:
df_acc = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])
df_f1 = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])
df_f1_ponderado = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])

In [ ]:
for classifier_name, classifier in classifiers:
    nova_linha = pd.DataFrame({'Classificador': [classifier_name], 'Rodada 1':[0] , 'Rodada 2':[0], 'Rodada 3':[0], 'Rodada 4':[0], 'Rodada 5':[0], 'Media':[0]})
    df_acc = pd.concat([df_acc, nova_linha], ignore_index=True)
    df_f1 = pd.concat([df_f1, nova_linha], ignore_index=True)
    df_f1_ponderado = pd.concat([df_f1_ponderado, nova_linha], ignore_index=True)

In [ ]:
df_f1

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


In [ ]:
df_f1_ponderado

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


In [ ]:
df_acc

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


 Usei como parâmetro para **average** o **'macro'** para o f1, e o **'weighted'** para o f1-ponderado

**'weighted'**:

Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

**'macro'**:

Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

 vide [Documentação oficial](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)

# Aplicando modelo de classificação

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support as score

def evaluate_model(model, X_test, y_test):
    # Predição dos rótulos
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

    # Cálculo da matriz de confusão
    cm = confusion_matrix(y_test, y_pred)

    # Cálculo da acurácia
    acc = accuracy_score(y_test, y_pred)

    # Cálculo do F1-score
    f1 = f1_score(y_test, y_pred, average='macro')

    # Cálculo do F1-score
    f1_poderado = f1_score(y_test, y_pred, average='weighted')

    # Outras métricas
    precision, recall, f1score, support = score(y_test, y_pred, average='macro')
    return cm, acc, f1, precision, recall, f1score, support, f1_poderado

In [ ]:
df_repvet

,0,1,2,3,4,5,6,7,8,9,...,393206,393207,393208,393209,393210,393211,393212,393213,393214,393215
0,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.253514,-0.368506,0.452481,-0.077948,-0.174519,0.397721,0.439606,0.283904,0.239466,0.868606
1,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.103246,-0.352199,0.157616,-0.666430,-0.755236,0.441820,0.579303,0.448379,0.324226,1.281295
2,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,-0.076338,-1.457131,0.513621,-0.785321,-0.698155,-0.144908,-0.032716,0.113876,0.423749,0.794382
3,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.208831,-1.140846,-0.148798,-1.034243,-0.268602,0.026068,0.578380,0.321614,0.406567,0.325306
4,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.381602,-0.865939,0.340151,-0.084570,-0.026963,0.801974,0.644628,0.329671,0.112665,0.851061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.218819,-0.662356,-0.900760,0.443632,-1.395365,0.030917,0.001971,1.138035,0.115485,1.609506
164,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.628640,0.706906,0.023609,0.772302,-1.505951,-0.658736,-0.689788,1.544010,-0.677629,0.837013
165,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.415427,-0.300990,0.774251,-0.609041,-0.432964,0.936262,0.529477,0.179168,-0.028599,0.790166
166,-0.196705,0.17107,0.107494,0.125283,-0.16889,-0.319846,0.27458,-0.134319,-0.265486,-0.019446,...,0.078053,-0.565056,0.281885,-0.548133,-0.682248,-0.154583,0.609861,0.535134,0.148935,0.412213


In [ ]:
classifiers

[('Multinomial Naive Bayes', MultinomialNB()),
 ('Complement Naive Bayes Classifier', ComplementNB()),
 ('KNN', KNeighborsClassifier()),
 ('SVM', SVC()),
 ('Random Forest', RandomForestClassifier(random_state=42)),
 ('AdaBoost', AdaBoostClassifier(random_state=42))]

In [ ]:
classificador=0
for classifier_name, classifier in classifiers:
    print('---', classifier_name, '---')
    y_true = []
    y_pred = []
    contador = 0
    serie_acc = pd.Series()
    serie_f1 = pd.Series()
    serie_f1_ponderado = pd.Series()


    for i in range(0, 5):  #Estou percorrendo as 5 rodadas
        train_index = kfold_train[i]
        test_index = kfold_test[i]
        contador +=1
        # df_repvet
        X_train_n, X_test_n = df_repvet.iloc[train_index], df_repvet.iloc[test_index]
        y_train, y_test = repvet_rotulado.iloc[train_index]['rotulo'], repvet_rotulado.iloc[test_index]['rotulo']


        scaler = MinMaxScaler()
        df_repvet_pos_treino = scaler.fit_transform(X_train_n)
        df_repvet_pos_teste =  scaler.transform(X_test_n)

        scaling_factor = 1000
        df_repvet_pos_i_treino = (df_repvet_pos_treino * scaling_factor).astype(int)
        X_train = pd.DataFrame(df_repvet_pos_i_treino, columns=df_repvet.columns)

        df_repvet_pos_i_teste = (df_repvet_pos_teste * scaling_factor).astype(int)
        X_test = pd.DataFrame(df_repvet_pos_i_teste, columns=df_repvet.columns)



        # Treinamento do modelo
        classifier.fit(X_train, y_train)
        cm, acc, f1, precision, recall, f1score, support,f1_poderado = evaluate_model(classifier, X_test, y_test)


        print(classifier_name + " Rodada " + str(contador) )
        print('Matriz de Confusão:')
        print(cm)
        print('Acurácia:', acc)
        print('F1-Score:', f1)
        print("outras métricas:")
        print('precision:', precision)
        print('recall:', recall)
        print('f1score:', f1score)
        print('support:', support)
        print('-------------------------------------')
        # serie_acc = serie_acc.append(pd.Series([acc]))
        serie_acc = pd.concat([serie_acc, pd.Series([acc])])
        # serie_f1 = serie_f1.append(pd.Series([f1]))
        serie_f1 = pd.concat([serie_f1, pd.Series([f1])])
        serie_f1_ponderado = pd.concat([serie_f1_ponderado, pd.Series([f1_poderado])])


    # Avaliação do modelo: Aqui estamos inserindo os valores das medias na serie
    media_acc = serie_acc[:5].mean()
    media_f1 = serie_f1[:5].mean()
    media_f1_ponderado = serie_f1_ponderado[:5].mean()
    # serie_acc = serie_acc.append(pd.Series([media_acc]))
    # serie_f1 = serie_f1.append(pd.Series([media_f1]))
    serie_acc = pd.concat([serie_acc, pd.Series([media_acc])])
    serie_f1 = pd.concat([serie_f1, pd.Series([media_f1])])
    serie_f1_ponderado = pd.concat([serie_f1_ponderado, pd.Series([media_f1_ponderado])])

    # print("Acurácia: " )
    # print(serie_acc)
    # print("F-1: " )
    # print(serie_f1)
    df_acc.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_acc.values
    df_f1.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_f1.values
    df_f1_ponderado.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_f1_ponderado.values
    classificador+=1
    print("=======================================================================================")
    # cm = confusion_matrix(y_true, y_pred)
    # acc = accuracy_score(y_true, y_pred)

--- Multinomial Naive Bayes ---


<ipython-input-130-ecc6b239afaf>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_acc = pd.Series()
<ipython-input-130-ecc6b239afaf>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1 = pd.Series()
<ipython-input-130-ecc6b239afaf>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1_ponderado = pd.Series()


              precision    recall  f1-score   support

           0       0.78      0.95      0.86        19
           1       0.00      0.00      0.00         8
           2       0.67      0.86      0.75         7

    accuracy                           0.71        34
   macro avg       0.48      0.60      0.54        34
weighted avg       0.57      0.71      0.63        34

Multinomial Naive Bayes Rodada 1
Matriz de Confusão:
[[18  1  0]
 [ 5  0  3]
 [ 0  1  6]]
Acurácia: 0.7058823529411765
F1-Score: 0.5357142857142857
outras métricas:
precision: 0.4830917874396135
recall: 0.6015037593984962
f1score: 0.5357142857142857
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.61      0.89      0.72        19
           1       0.00      0.00      0.00         8
           2       0.80      0.57      0.67         7

    accuracy                           0.62        34
   macro avg       0.47      0.49      0.46  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.67      0.95      0.78        19
           1       0.00      0.00      0.00         8
           2       0.86      0.86      0.86         7

    accuracy                           0.71        34
   macro avg       0.51      0.60      0.55        34
weighted avg       0.55      0.71      0.61        34

Multinomial Naive Bayes Rodada 3
Matriz de Confusão:
[[18  0  1]
 [ 8  0  0]
 [ 1  0  6]]
Acurácia: 0.7058823529411765
F1-Score: 0.546583850931677
outras métricas:
precision: 0.5079365079365079
recall: 0.6015037593984962
f1score: 0.546583850931677
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.70      1.00      0.83        19
           1       1.00      0.12      0.22         8
           2       0.80      0.67      0.73         6

    accuracy                           0.73        33
   macro avg       0.83      0.60      0.59    

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.70      1.00      0.83        19
           1       0.00      0.00      0.00         8
           2       0.67      0.67      0.67         6

    accuracy                           0.70        33
   macro avg       0.46      0.56      0.50        33
weighted avg       0.53      0.70      0.60        33

Random Forest Rodada 4
Matriz de Confusão:
[[19  0  0]
 [ 6  0  2]
 [ 2  0  4]]
Acurácia: 0.696969696969697
F1-Score: 0.4975845410628019
outras métricas:
precision: 0.4567901234567901
recall: 0.5555555555555555
f1score: 0.4975845410628019
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.65      0.94      0.77        18
           1       0.00      0.00      0.00         9
           2       0.67      0.67      0.67         6

    accuracy                           0.64        33
   macro avg       0.44      0.54      0.48        33
we

<ipython-input-130-ecc6b239afaf>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_acc = pd.Series()
<ipython-input-130-ecc6b239afaf>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1 = pd.Series()
<ipython-input-130-ecc6b239afaf>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1_ponderado = pd.Series()


              precision    recall  f1-score   support

           0       0.71      0.89      0.79        19
           1       0.33      0.12      0.18         8
           2       0.57      0.57      0.57         7

    accuracy                           0.65        34
   macro avg       0.54      0.53      0.51        34
weighted avg       0.59      0.65      0.60        34

AdaBoost Rodada 1
Matriz de Confusão:
[[17  2  0]
 [ 4  1  3]
 [ 3  0  4]]
Acurácia: 0.6470588235294118
F1-Score: 0.5146481425551193
outras métricas:
precision: 0.5376984126984127
recall: 0.5303884711779449
f1score: 0.5146481425551193
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.57      0.63      0.60        19
           1       0.08      0.12      0.10         8
           2       1.00      0.14      0.25         7

    accuracy                           0.41        34
   macro avg       0.55      0.30      0.32        34
weight

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
df_acc

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.705882,0.617647,0.705882,0.727273,0.606061,0.672549
1,Complement Naive Bayes Classifier,0.735294,0.617647,0.705882,0.69697,0.606061,0.672371
2,KNN,0.705882,0.676471,0.647059,0.69697,0.545455,0.654367
3,SVM,0.735294,0.647059,0.705882,0.69697,0.636364,0.684314
4,Random Forest,0.735294,0.617647,0.735294,0.69697,0.636364,0.684314
5,AdaBoost,0.647059,0.411765,0.617647,0.666667,0.575758,0.583779


In [ ]:
df_f1

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.535714,0.463357,0.546584,0.591861,0.499344,0.527372
1,Complement Naive Bayes Classifier,0.537879,0.446263,0.546584,0.497585,0.438538,0.49337
2,KNN,0.535714,0.602564,0.550117,0.575758,0.443902,0.541611
3,SVM,0.537879,0.472222,0.546584,0.511928,0.455319,0.504786
4,Random Forest,0.537879,0.451505,0.626455,0.497585,0.479798,0.518644
5,AdaBoost,0.514648,0.316667,0.45,0.509091,0.341615,0.426404


In [ ]:
df_f1_ponderado

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.633403,0.54151,0.613811,0.661729,0.556969,0.601484
1,Complement Naive Bayes Classifier,0.637032,0.530952,0.613811,0.596838,0.509816,0.57769
2,KNN,0.633403,0.653949,0.596462,0.641322,0.508204,0.606668
3,SVM,0.637032,0.556373,0.613811,0.597738,0.526886,0.586368
4,Random Forest,0.637032,0.53974,0.675817,0.596838,0.5427,0.598425
5,AdaBoost,0.602288,0.410294,0.548529,0.592837,0.481084,0.527007
